In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

In [2]:
#https://www.kaggle.com/datasets/altavish/boston-housing-dataset
df = pd.read_csv('../HousingData.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [3]:
df.fillna({'CRIM':df['CRIM'].median()},inplace=True)
df.fillna({'ZN':df['ZN'].median()},inplace=True)
df.fillna({'INDUS':df['INDUS'].median()},inplace=True)
df.fillna({'AGE':df['AGE'].median()},inplace=True)
df.fillna({'LSTAT':df['LSTAT'].median()},inplace=True)

df.fillna({'CHAS':df['CHAS'].mode()[0]},inplace=True)

In [4]:
df.rename(columns={'MEDV': 'price'}, inplace=True)

In [5]:
X = df.drop(columns=['price'])
y= df['price']

X_train, X_vt, y_train, y_vt = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_vt, y_vt, test_size=0.4, random_state=42
)

In [6]:
class RidgeRegression:
    """
    Класс для реализации регрессии Риджа, которая включает регуляризацию для предотвращения переобучения.

    Параметры:
        reg (float): Параметр регуляризации. По умолчанию 0.001.
    """

    def __init__(self, reg=0.001):
        # инициализация параметра регуляризации
        self.reg = reg

    def fit(self, X, y):
        """
        Обучает модель регрессии Риджа на данных.

        Параметры:
            X (numpy.ndarray): Матрица признаков.
            y (numpy.ndarray): Вектор целевых значений.
        """
        V, D, U = np.linalg.svd(X, full_matrices=False)  # разложение матрицы признаков с использованием SVD
        w = D / (self.reg + D ** 2)  # вычисление весов с учетом регуляризации
        self.coef_ = U.T @ (w * (V.T @ y))  # вычисление коэффициентов модели

    def predict(self, X):
        """
        Предсказывает целевые значения для новых данных.

        Параметры:
            X (numpy.ndarray): Матрица признаков.

        Возвращает:
            numpy.ndarray: Предсказанные целевые значения.
        """
        return X @ self.coef_  # вычисление предсказанных значений

    def optimize_regularization(self, X_train, X_test, y_train, y_test, reg_range=(0, 1, 0.001)):
        """
        Оптимизирует параметр регуляризации на основе метрик на тестовых данных.

        Параметры:
            X_train (numpy.ndarray): Матрица признаков для обучения.
            X_test (numpy.ndarray): Матрица признаков для тестирования.
            y_train (numpy.ndarray): Вектор целевых значений для обучения.
            y_test (numpy.ndarray): Вектор целевых значений для тестирования.
            reg_range (tuple): Диапазон значений для параметра регуляризации. По умолчанию (0, 1, 0.001).

        Возвращает:
            dict: Словарь с лучшим значением параметра регуляризации.
        """
        reg = np.arange(*reg_range)  # создание массива значений параметра регуляризации
        metrics = {
            'mse': []  # список для хранения значений метрики MSE
        }

        for i in reg:
            self.reg = i  # установка текущего значения параметра регуляризации
            self.fit(X_train, y_train)  # обучение модели
            y_pred = self.predict(X_test)  # предсказание значений
            mse = self._mse(y_test, y_pred)  # вычисление метрики MSE
            metrics['mse'].append(mse)  # добавление значения метрики в список

        best_index = np.argmin(metrics['mse'])  # индекс лучшего значения параметра регуляризации
        self.reg = reg[best_index]  # установка лучшего значения параметра регуляризации
        return {'best_reg': self.reg}  # возвращение лучшего значения параметра регуляризации

    @staticmethod
    def _mse(y_true, y_pred):
        """
        Вычисляет среднеквадратичную ошибку (MSE).

        Параметры:
            y_true (numpy.ndarray): Вектор истинных целевых значений.
            y_pred (numpy.ndarray): Вектор предсказанных целевых значений.

        Возвращает:
            float: Значение метрики MSE.
        """
        return np.mean((y_true - y_pred) ** 2)  # вычисление MSE

    @staticmethod
    def _mae(y_true, y_pred):
        """
        Вычисляет среднюю абсолютную ошибку (MAE).

        Параметры:
            y_true (numpy.ndarray): Вектор истинных целевых значений.
            y_pred (numpy.ndarray): Вектор предсказанных целевых значений.

        Возвращает:
            float: Значение метрики MAE.
        """
        return np.mean(np.abs(y_true - y_pred))  # вычисление MAE

    @staticmethod
    def _r2(y_true, y_pred):
        """
        Вычисляет коэффициент детерминации (R^2).

        Параметры:
            y_true (numpy.ndarray): Вектор истинных целевых значений.
            y_pred (numpy.ndarray): Вектор предсказанных целевых значений.

        Возвращает:
            float: Значение метрики R^2.
        """
        ss_total = np.sum((y_true - np.mean(y_true)) ** 2)  # вычисление общей суммы квадратов
        ss_residual = np.sum((y_true - y_pred) ** 2)  # вычисление суммы квадратов остатков
        return 1 - (ss_residual / ss_total)  # вычисление R^2

In [7]:
rr = RidgeRegression()

In [8]:
%time rr.optimize_regularization(X_train, X_val, y_train, y_val, reg_range=(0, 100, 0.01))

CPU times: user 3.71 s, sys: 7.27 ms, total: 3.72 s
Wall time: 3.77 s


{'best_reg': 19.21}

In [9]:
%time rr.fit(X_train, y_train)
y_pred = rr.predict(X_test)

mse = rr._mse(y_test, y_pred)
mae = rr._mae(y_test, y_pred)
r2 = rr._r2(y_test, y_pred)

print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R^2: {r2:.4f}")

CPU times: user 2.04 ms, sys: 9 µs, total: 2.05 ms
Wall time: 1.39 ms
MSE: 32.5970
MAE: 3.7292
R^2: 0.6423


In [10]:
model = Ridge(alpha=rr.reg, fit_intercept=False)
%time model.fit(X_train, y_train)
y_pred_sklearn = model.predict(X_test)

mse_sklearn = rr._mse(y_test, y_pred_sklearn)
mae_sklearn = rr._mae(y_test, y_pred_sklearn)
r2_sklearn = rr._r2(y_test, y_pred_sklearn)

print(f"MSE: {mse_sklearn:.4f}")
print(f"MAE: {mae_sklearn:.4f}")
print(f"R^2: {r2_sklearn:.4f}")

CPU times: user 2.85 ms, sys: 1.34 ms, total: 4.19 ms
Wall time: 58.3 ms
MSE: 32.5970
MAE: 3.7292
R^2: 0.6423
